# **Hybrid Pipeline**

**✔ Whisper-medium (local) for ASR**

**✔ MarianMT / mBART50 for Translation**

This gives far better Arabic→English accuracy than Whisper-medium’s built-in translation.

Whisper-medium is excellent at Arabic speech recognition,
but weak at translation.

### **So we split tasks:**

ASR (Arabic speech → Arabic text) → Whisper medium

Translation (Arabic text → English text) → MarianMT/mBART (HuggingFace)


### **This outperforms:**

Whisper medium translation

Whisper large translation on CPU

Whisper large on weak GPUs

Legacy Kaldi/Vosk Arabic models

In [19]:
!pip install git+https://github.com/openai/whisper.git
!pip install transformers sentencepiece torch pydub
!apt-get install -y ffmpeg


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-hu03_p7s
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-hu03_p7s
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [20]:
import whisper
from pydub import AudioSegment

whisper_model = whisper.load_model("medium")

def preprocess_audio(path, out="temp.wav"):
    audio = AudioSegment.from_file(path)
    audio = audio.set_frame_rate(16000).set_channels(1)
    audio.export(out, format="wav")
    return out

def speech_to_arabic_text(audio_path):
    processed = preprocess_audio(audio_path)
    result = whisper_model.transcribe(processed, language="ar")
    return result["text"].strip()


In [21]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-ar-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
translator_model = MarianMTModel.from_pretrained(model_name)

def arabic_to_english(text):
    tokens = tokenizer(text, return_tensors="pt", padding=True)
    translated = translator_model.generate(**tokens)
    return tokenizer.decode(translated[0], skip_special_tokens=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [22]:
def arabic_voice_to_english(audio_path):
    ar_text = speech_to_arabic_text(audio_path)
    en_text = arabic_to_english(ar_text)
    return en_text


In [25]:
query = arabic_voice_to_english("arabic_test.mp3")
print("Final English Query:", query)

#English: What is the college's policy if a student fails a course? And does he have to repeat the course in the following semester?

Final English Query: What's the college policy if the student fails a bat, and does he have to return the adhesive in the next chapter?


In [26]:
query = arabic_voice_to_english("arabic_sample1.mp3")
print("Final English Query:", query)

# English: Is a student allowed to retake an exam if they are absent for health reasons? What are the required procedures?

Final English Query: Would a student be allowed to retest if absent for health reasons and what procedures are required?


In [27]:
query = arabic_voice_to_english("arabic_sample2.mp3")
print("Final English Query:", query)

#English: What is the required attendance percentage to pass the course? And can absences be made up?

Final English Query: What percentage of attendance is required to succeed in the substance? Can absence be compensated?


In [28]:
query = arabic_voice_to_english("arabic_sample3.mp3")
print("Final English Query:", query)

#English: Is a student allowed to postpone the semester? And what documents are required for approval?

Final English Query: Is the student allowed to postpone the semester? What documents are required for approval?


Hybrid pipeline proved way better than local whisper medium model alone.
*  The test audio fails significantly.
*  It generates sample1 and sample3 100% correctly.
*  For sample2, it fails only for 1 word, it takes course as substance, rest of the sentence is correctly translated.


